In [1]:
import numpy as np
import matplotlib.pyplot as plt
from emitter import Emitter

In [32]:
cmin = 0
cmax = 2
n_points = 20

In [33]:
c = np.linspace(cmin, cmax, n_points)

In [34]:
grid = np.array([(c[i], c[j]) for j in range(n_points) for i in range(n_points)])

In [35]:
y = lambda x: .2 * x ** 3 - .3 * x + .3
fx = y(c)

In [36]:
e = Emitter(cmin, cmax, pmax=1, n_coords=2)
e.set_origin(np.array([1, 2]))
particles = [e.give()]

In [113]:
class Cube:
    def __init__(self, coords, state="empty"):
        """ Cube is defined by a down-bottom-left point """
        self.c = coords.copy()
        self.state = state

    def __repr__(self):
        return f"[{self.coords.round(2)}] - {self.state}"

    @property
    def coords(self):
        return self.c

    @coords.setter
    def coords(self, val):
        self.c = val

    def is_empty(self):
        return self.state == 'empty'

class HeightMap:
    def __init__(self, cmin, cmax, n_points, n_dims=2, estimator=None):
        self.n = n_points
        self.n_dims = n_dims
        self.hmap = np.empty((self.n ** n_dims,), dtype=object)
        
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = c[1] - c[0]
        
        for i in range(self.n ** n_dims):
            idx = divmod(i, n_points)
            cp = np.array([c[_] for _ in idx])
            self.hmap[i] = Cube(cp, 'terrain' if self.under_f(y, cp) else 'empty')
    
    def dn(self, params):
        """ n-dimentional index convertion to 1D array """
        n = len(params)
        return self.hmap[sum([_ * self.n ** p for _, p in zip(params, range(n - 1, -1, -1))])]
        
    
    def i_to_ndim_idx(i):
        return tuple([i])
    
    def under_f(self, f, point):
        f_val = f(point[:-1])
        return point[-1] <= f_val

In [114]:
hmap = HeightMap(0, 2, 20)
print(hmap.hmap.shape)

(400,)


In [115]:
hmap.dn((19, 19))

[[2. 2.]] - empty

In [116]:
class Simulation:
    def __init__(self, cmin=0, cmax=2, n_points=20):
        self.particles = []
        self.dt = 0.003
        self.cmin = cmin
        self.cmax = cmax
        self.n_grid = n_points
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = self.c[1] - self.c[0]
    
    def add_particles(self, particles):
        self.particles.extend(particles)
    
    def update_particles(self):
        # 1) gravity
        for particle in self.particles:
            print("Discrete:", self.discritize(particle))
            particle.grad[((particle.coords + particle.grad) < self.cmin) | ((particle.coords + particle.grad) > self.cmax)] = 0
            particle.coords += particle.grad
            particle.grad[-1] += -9.81 * self.dt
    
    def discritize(self, particle):
        return [int(_ / self.step) for _ in particle.coords]
    
    def reset(self):
        self.particles = []

In [117]:
s = Simulation()
s.add_particles(particles)
s.particles

[Particle [[1. 2.]] -> [0. 0.]]

In [121]:
import ipywidgets as widgets

fig, ax = plt.subplots()

@widgets.interact(t=widgets.FloatSlider(min=0.0, max=2.0, value=0), continuous_update=False)
def f(t):
    if t == 0:
        s.reset()
        s.add_particles([_.copy() for _ in particles])
    s.update_particles()
    col = ['green' if hmap.hmap[i].state == 'terrain' else 'blue' for i in range(hmap.n ** 2)]
    coords = np.array([hmap.hmap[i].coords for i in range(hmap.n ** 2)])
    plt.scatter(coords[:, 0], coords[:, 1], s=1, c=col, label="terrain");
    plt.scatter([p.x for p in s.particles], [p.y for p in s.particles], c='blue', label="particle")
    plt.plot(c, fx, c='orange', label="Surface function");
    plt.xlim(cmin, cmax);
    plt.ylim(cmin, cmax);
    plt.legend(loc="upper left");

interactive(children=(FloatSlider(value=0.0, description='t', max=2.0), Output()), _dom_classes=('widget-inter…